In [6]:
from sklearn.preprocessing import StandardScaler
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import mysql.connector
import pandas as pd

import json

with open('config.json') as f:
    config = json.load(f)


cnx = mysql.connector.connect(user=config['user'],
                              password=config['password'],
                              host=config['host'],
                              database=config['database'])

cursor = cnx.cursor()

# Example query
query = ("SELECT * FROM PH_Temp_Diet")

cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()

# Create a pandas DataFrame
data = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])


cursor.close()
cnx.close()

In [4]:
df3 =data
df3 = df3.drop(['id'], axis=1)

# Pivot the data to create columns for each unique food_name
pivoted_data = df3.pivot_table(index=['person_id', 'date','PH','temperature'], columns='food_name', values='amount')
pivoted_data
# Rename columns to remove the 'food_name' label prefix
pivoted_data.columns.name = None
pivoted_data.columns = pivoted_data.columns.str.replace('food_name_', '')

# # Join the pivoted data with the original data on the 'person_id' and 'date' columns
joined_data = pd.merge(data.drop(columns=['food_name', 'amount','id']), pivoted_data, on=['person_id', 'date','temperature','PH'])

# # Save the joined data to a new CSV file
#df3 = pivoted_data
# Group by 'person_id' and 'date', and aggregate the other columns
grouped_df = joined_data.groupby(['person_id', 'date','temperature','PH'], as_index=False);
joined_data = joined_data.drop_duplicates()
joined_data

,PH,temperature,date,person_id,Apple,Apple Pie,Banana,Beer,Braised Potatoes,Bread,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,5.40,35.8,06.09.2017,1,0.0,0.0,100.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,150.0
97,6.03,36.1,07.09.2017,1,0.0,0.0,0.0,0.0,0.0,180.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,50.0
190,5.81,36.4,08.09.2017,1,0.0,0.0,0.0,0.0,0.0,250.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,50.0
283,6.10,36.6,09.09.2017,1,0.0,0.0,110.0,500.0,0.0,100.0,...,0.0,0.0,0.0,60.0,0.0,0.0,400.0,0.0,0.0,0.0
376,6.06,35.9,10.09.2017,1,0.0,0.0,100.0,0.0,0.0,150.0,...,0.0,0.0,0.0,30.0,0.0,0.0,300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19441,5.82,36.5,03.04.2018,1,140.0,0.0,78.0,0.0,0.0,43.0,...,0.0,0.0,0.0,0.0,0.0,65.0,1050.0,0.0,0.0,0.0
19534,5.03,36.1,04.04.2018,1,214.0,0.0,0.0,0.0,0.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0
19627,5.59,36.3,05.04.2018,1,0.0,0.0,0.0,0.0,0.0,190.0,...,0.0,250.0,0.0,34.0,0.0,0.0,1350.0,0.0,0.0,0.0
19720,5.40,35.8,06.09.2017,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Define a function to recommend food items based on pH level
def recommend_foods(ph_level, df):
    # Calculate the mean pH for each food item
    food_phs = df.iloc[:, 5:].mean()
    # Determine which food items are associated with low or high pH levels
    low_phs = food_phs[food_phs < ph_level].index.tolist()
    high_phs = food_phs[food_phs > ph_level].index.tolist()
    
    # Return the recommended food items
    if ph_level < 7:
        return high_phs
    else:
        return low_phs

In [10]:
recommend_foods(6.8,joined_data)

['Banana',
 'Beer',
 'Braised Potatoes',
 'Bread',
 'Cake',
 'Cheese',
 'Cookie',
 'Croissant',
 'Curd',
 'Curd Pie',
 'Dried apricots',
 'Fried potatoes',
 'Fruit tea',
 'Ice cream',
 'Latte',
 'Lemon water',
 'Liquid',
 'Mashed potatoes',
 'Mayonnaise salad',
 'Meat',
 'Meat pilaf',
 'Meatless pilaf',
 'Melon',
 'Mineral water (Borjomi)',
 'Mineral water (Esentuki - 4)',
 'Mineral water (Prolom)',
 'Morse',
 'Noodles with shrimp',
 'Oatmeal',
 'Orange juice',
 'Patty with cabbage',
 'Peach',
 'Pear',
 'Pizza with meat',
 'Plum',
 'Potato dumplings',
 'Pumpkin seeds',
 'Red wine',
 'Scrambled eggs',
 'Sushi',
 'Tangerines',
 'Tea(black/green)',
 'The nuts',
 'Vegetable salad',
 'Water',
 'Watermelon',
 'White wine',
 'Zucchini Fritters']

In [37]:
df2= data
df2.head(5)

,id,PH,temperature,food_name,amount,date,person_id
0,39446,5.4,35.8,Gym,0.0,06.09.2017,1
1,39447,5.4,35.8,Liquid,1300.0,06.09.2017,1
2,39448,5.4,35.8,Lemon water,250.0,06.09.2017,1
3,39449,5.4,35.8,Tea(black/green),0.0,06.09.2017,1
4,39450,5.4,35.8,Fruit tea,600.0,06.09.2017,1


In [3]:
import pandas as pd

# Load data from your database or CSV file
df3 = data

df3 = df3.drop(['id'], axis=1)

# Pivot the data to create columns for each unique food_name
pivoted_data = df3.pivot_table(index=['person_id', 'date','PH','temperature'], columns='food_name', values='amount')
pivoted_data
# Rename columns to remove the 'food_name' label prefix
pivoted_data.columns.name = None
pivoted_data.columns = pivoted_data.columns.str.replace('food_name_', '')

# # Join the pivoted data with the original data on the 'person_id' and 'date' columns
joined_data = pd.merge(data.drop(columns=['food_name', 'amount','id']), pivoted_data, on=['person_id', 'date','temperature','PH'])

# # Save the joined data to a new CSV file
#df3 = pivoted_data
# Group by 'person_id' and 'date', and aggregate the other columns
grouped_df = joined_data.groupby(['person_id', 'date','temperature','PH'], as_index=False);
joined_data = joined_data.drop_duplicates()
joined_data

,PH,temperature,date,person_id,Apple,Apple Pie,Banana,Beer,Braised Potatoes,Bread,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,5.40,35.8,06.09.2017,1,0.0,0.0,100.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,150.0
97,6.03,36.1,07.09.2017,1,0.0,0.0,0.0,0.0,0.0,180.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,50.0
190,5.81,36.4,08.09.2017,1,0.0,0.0,0.0,0.0,0.0,250.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,50.0
283,6.10,36.6,09.09.2017,1,0.0,0.0,110.0,500.0,0.0,100.0,...,0.0,0.0,0.0,60.0,0.0,0.0,400.0,0.0,0.0,0.0
376,6.06,35.9,10.09.2017,1,0.0,0.0,100.0,0.0,0.0,150.0,...,0.0,0.0,0.0,30.0,0.0,0.0,300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19441,5.82,36.5,03.04.2018,1,140.0,0.0,78.0,0.0,0.0,43.0,...,0.0,0.0,0.0,0.0,0.0,65.0,1050.0,0.0,0.0,0.0
19534,5.03,36.1,04.04.2018,1,214.0,0.0,0.0,0.0,0.0,150.0,...,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0
19627,5.59,36.3,05.04.2018,1,0.0,0.0,0.0,0.0,0.0,190.0,...,0.0,250.0,0.0,34.0,0.0,0.0,1350.0,0.0,0.0,0.0
19720,5.40,35.8,06.09.2017,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from sklearn.model_selection import train_test_split

X= joined_data.iloc[:,0:2]
X

,PH,temperature
0,5.40,35.8
97,6.03,36.1
190,5.81,36.4
283,6.10,36.6
376,6.06,35.9
...,...,...
19441,5.82,36.5
19534,5.03,36.1
19627,5.59,36.3
19720,5.40,35.8


In [5]:
y = joined_data.iloc[:,5:]
y

,Apple Pie,Banana,Beer,Braised Potatoes,Bread,Buckwheat porridge,Bun,Cake,Candies,Cheese,...,Tangerines,Tea(black/green),Tequila,The nuts,The vinaigrette,Vegetable salad,Water,Watermelon,White wine,Zucchini Fritters
0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,150.0
97,0.0,0.0,0.0,0.0,180.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40.0,0.0,0.0,300.0,0.0,0.0,50.0
190,0.0,0.0,0.0,0.0,250.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,50.0
283,0.0,110.0,500.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,60.0,0.0,0.0,400.0,0.0,0.0,0.0
376,0.0,100.0,0.0,0.0,150.0,0.0,150.0,0.0,0.0,0.0,...,0.0,0.0,0.0,30.0,0.0,0.0,300.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19441,0.0,78.0,0.0,0.0,43.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,65.0,1050.0,0.0,0.0,0.0
19534,0.0,0.0,0.0,0.0,150.0,0.0,0.0,69.0,0.0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0
19627,0.0,0.0,0.0,0.0,190.0,0.0,0.0,55.0,0.0,29.0,...,0.0,250.0,0.0,34.0,0.0,0.0,1350.0,0.0,0.0,0.0
19720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
y = data.groupby(['person_id', 'date'])[['food_name', 'amount']].agg(list)
# Reset the index
y = y.reset_index()

# Custom function to combine food_name and amount into one list
def combine_food_and_amount(food_names, amounts):
    combined = []
    for food, amount in zip(food_names, amounts):
        combined.append((food, amount))
    return combined

# Apply the custom function to combine food_name and amount columns
y['food_and_amount'] = y.apply(lambda row: combine_food_and_amount(row['food_name'], row['amount']), axis=1)

# Drop the food_name and amount columns
y = y.drop(['food_name', 'amount','person_id','date'], axis=1)

# Print the modified DataFrame
print(y.head(5))

                                     food_and_amount
0  [(Gym, 0.0), (Liquid, 1250.0), (Lemon water, 2...
1  [(Gym, 0.0), (Liquid, 2500.0), (Lemon water, 3...
2  [(Gym, 1.0), (Liquid, 3050.0), (Lemon water, 3...
3  [(Gym, 1.0), (Liquid, 2900.0), (Lemon water, 3...
4  [(Gym, 1.0), (Liquid, 1750.0), (Lemon water, 2...


In [30]:
import pandas as pd

# Sample input data
input_data = pd.DataFrame(y)

# Function to process a single row
def process_row(row):
    return ', '.join([f'{food}: {amount}' for food, amount in row['food_and_amount']])

# Process each row and store the result in a new column
input_data['food_and_amount_str'] = input_data.apply(process_row, axis=1)

# Drop the original food_and_amount column
y = input_data.drop(['food_and_amount'], axis=1)

print(y)

                                   food_and_amount_str
0    Gym: 0.0, Liquid: 1250.0, Lemon water: 250.0, ...
1    Gym: 0.0, Liquid: 2500.0, Lemon water: 300.0, ...
2    Gym: 1.0, Liquid: 3050.0, Lemon water: 300.0, ...
3    Gym: 1.0, Liquid: 2900.0, Lemon water: 300.0, ...
4    Gym: 1.0, Liquid: 1750.0, Lemon water: 250.0, ...
..                                                 ...
209  Gym: 0.0, Liquid: 3195.0, Lemon water: 300.0, ...
210  Gym: 0.0, Liquid: 1200.0, Lemon water: 250.0, ...
211  Gym: 0.0, Liquid: 2050.0, Lemon water: 250.0, ...
212                                           Gym: 0.0
213  Liquid: 100.0, Tea(black/green): 122.0, Minera...

[214 rows x 1 columns]


In [19]:
y.shape

(214, 1)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.6, random_state=42)
y_train

,food_and_amount_str
64,"Gym: 0.0, Liquid: 2920.0, Lemon water: 300.0, ..."
44,"Gym: 0.0, Liquid: 2825.0, Lemon water: 300.0, ..."
139,"Gym: 0.0, Liquid: 1250.0, Lemon water: 250.0, ..."
40,"Gym: 1.0, Liquid: 1400.0, Lemon water: 250.0, ..."
123,"Gym: 0.0, Liquid: 1550.0, Lemon water: 250.0, ..."
...,...
106,"Gym: 0.0, Liquid: 3450.0, Lemon water: 300.0, ..."
14,"Gym: 0.0, Liquid: 1250.0, Lemon water: 250.0, ..."
92,"Gym: 0.0, Liquid: 3275.0, Lemon water: 300.0, ..."
179,"Gym: 1.0, Liquid: 1400.0, Lemon water: 250.0, ..."


In [32]:
from sklearn.linear_model import LinearRegression
# Train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = lr.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# print("MSE:", mse)

ValueError: could not convert string to float: 'Gym: 0.0, Liquid: 2920.0, Lemon water: 300.0, Tea(black/green): 0.0, Fruit tea: 520.0, Water: 1650.0, Latte: 0.0, Mineral water (Esentuki - 4): 250.0, Mineral water (Borjomi): 0.0, Mineral water (Prolom): 0.0, Tequila: 200.0, Red wine: 0.0, White wine: 0.0, Strong alcohol: 0.0, Beer: 0.0, Morse: 0.0, Milk: 0.0, Orange juice: 0.0, Oatmeal: 0.0, Pizza with meat: 0.0, Cheese Pizza: 0.0, Cheesecakes: 0.0, Scrambled eggs: 180.0, Mayonnaise salad: 0.0, Bread: 70.0, Meat: 0.0, Sauce: 0.0, Cutlet: 0.0, Sausage: 0.0, Fish: 0.0, Sushi: 358.0, Cod liver: 0.0, Cheese: 0.0, Curd: 0.0, Sour cream: 0.0, Buckwheat porridge: 0.0, Noodles with vegetables: 0.0, Noodles with shrimp: 0.0, Meat pilaf: 0.0, Meatless pilaf: 0.0, Pea soup with meat: 0.0, Fried vegetables: 0.0, Pyagse with vegetables: 0.0, Dumplings with cabbage: 0.0, Potato dumplings: 0.0, Zucchini Fritters: 0.0, Stewed cabbage: 0.0, Solyanka with meat: 0.0, Eggplant with cheese: 0.0, Pasta: 0.0,

In [34]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Replace this with your actual dataset
data = pd.DataFrame({
    'pH_level': [5.0, 6.0, 7.0, 5.5],
    'food_and_activities': [
        [('Gym', 0.0), ('Liquid', 2920.0), ('Lemon water', 300.0)],
        [('Gym', 1.0), ('Liquid', 2000.0), ('Lemon water', 100.0)],
        [('Gym', 0.0), ('Liquid', 2500.0), ('Lemon water', 0.0)],
        [('Gym', 1.0), ('Liquid', 1800.0), ('Lemon water', 200.0)],
    ]
})

def categorize_pH(pH):
    if pH < 7.0:
        return "acidic"
    elif pH == 7.0:
        return "neutral"
    else:
        return "alkaline"

data['pH_category'] = data['pH_level'].apply(categorize_pH)

def binary_representation(row):
    binary_list = [row['pH_category']]
    for food, amount in row['food_and_activities']:
        if amount > 0:
            binary_list.append(food)
    return binary_list

data['binary'] = data.apply(binary_representation, axis=1)

transactions = data['binary'].tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
binary_df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(binary_df, min_support=0.5, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# Filter the rules based on the antecedent (left-hand side) containing the pH category "acidic"
acidic_rules = rules[rules['antecedents'].apply(lambda x: 'acidic' in x)]

print(acidic_rules)


                      antecedents                 consequents  \
5                        (acidic)                       (Gym)   
8                        (acidic)               (Lemon water)   
11                       (acidic)                    (Liquid)   
18                  (Gym, acidic)               (Lemon water)   
20          (acidic, Lemon water)                       (Gym)   
22                       (acidic)          (Gym, Lemon water)   
25                  (Gym, acidic)                    (Liquid)   
26               (Liquid, acidic)                       (Gym)   
29                       (acidic)               (Gym, Liquid)   
30               (Liquid, acidic)               (Lemon water)   
32          (acidic, Lemon water)                    (Liquid)   
34                       (acidic)       (Liquid, Lemon water)   
36          (Gym, Liquid, acidic)               (Lemon water)   
38     (Gym, acidic, Lemon water)                    (Liquid)   
39  (Liquid, acidic, Lemo

In [39]:
def recommend_food_and_activities(pH_level, rules):
    pH_category = categorize_pH(pH_level)

    # Filter the rules based on the antecedent (left-hand side) containing the pH category
    recommendations = rules[rules['antecedents'].apply(lambda x: pH_category in x)]

    # Sort the rules by confidence and select the top 5 rules
    top_recommendations = recommendations.sort_values(by='confidence', ascending=False).head(5)

    # Extract the consequents (right-hand side) of the rules
    recommended_items = set()
    for _, row in top_recommendations.iterrows():
        recommended_items.update(row['consequents'])

    # Remove the pH category from the recommended items
    recommended_items.discard(pH_category)

    return list(recommended_items)


# Example usage
urine_pH = 2  # Replace this with the actual pH level data from the sensor
recommended_items = recommend_food_and_activities(urine_pH, acidic_rules)
print("Recommended food and activities:", recommended_items)


Recommended food and activities: ['Liquid', 'Lemon water']


In [40]:
import joblib

# Save the association rules DataFrame to a file
joblib.dump(acidic_rules, 'acidic_rules.joblib')

# Save the categorize_pH function to a file
joblib.dump(categorize_pH, 'categorize_pH_function.joblib')

['categorize_pH_function.joblib']

In [41]:
import joblib

# Load the association rules DataFrame and the categorize_pH function
acidic_rules = joblib.load('acidic_rules.joblib')
categorize_pH = joblib.load('categorize_pH_function.joblib')

# Use the loaded model and function to make recommendations
urine_pH = 5.5  # Replace this with the actual pH level data from the sensor
recommended_items = recommend_food_and_activities(urine_pH, acidic_rules)
print("Recommended food and activities:", recommended_items)


Recommended food and activities: ['Liquid', 'Lemon water']
